<a href="https://colab.research.google.com/github/captaindeadpool53/songSourceSeperator/blob/main/notebooks/SongSourceSeperator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# prompt: I want to use training data from my google drive and then save a dictionary on dricve

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import os
from src.data_preprocessing.datasetHandler import DatasetHandler
from src.evaluation.evaluationHandler import EvaluationHandler
from src.model_architectures.unet import UNET
from config.constants import Constants
import tensorflow as tf

In [26]:
FRAME_SIZE =2048   #512
HOP_LENGTH = 512   #256
SEGMENT_LENGTH_IN_SECONDS = 2
SAMPLE_RATE = 16000
# MONO = True
DATA_ROOT_PATH = '/content/drive/MyDrive/Colab Notebooks/babySlakh/babyslakh_16k'
datasetHandler = DatasetHandler(DATA_ROOT_PATH, SAMPLE_RATE, SEGMENT_LENGTH_IN_SECONDS, FRAME_SIZE, HOP_LENGTH)
datasetHandler.loadAndPreprocessData()
trainingDataset, testDataset = datasetHandler.getDatasets()
inputShape, numberOfOutputChannels = datasetHandler.getShapeData()
if os.path.exists(Constants.CHECKPOINT_PATH.value):
    unetModel = tf.keras.models.load_model(Constants.CHECKPOINT_PATH.value)
else:
    unetModel = UNET(inputShape, numberOfOutputChannels)
    optimizer = tf.keras.optimizers.AdamW(weight_decay=1e-6, learning_rate=1e-3)
    unetModel.compile(loss = EvaluationHandler.drumsLossFunction, optimizer = optimizer)

    learningRateSchedulerCallback = tf.keras.callbacks.LearningRateScheduler(EvaluationHandler.learningRateScheduler)

    checkpointCallback = tf.keras.callbacks.ModelCheckpoint(
        filepath=Constants.CHECKPOINT_PATH.value,
        save_weights_only=False,
        save_best_only=True,
        monitor="val_loss",
        verbose=1,
    )
    callbacks = [checkpointCallback, learningRateSchedulerCallback]
    unetModel.fit(
        trainingDataset,
        validation_data = testDataset,
        callbacks=callbacks,
        batch_size=Constants.BATCH_SIZE.value,
        epochs=40,
        verbose=1
    )

::: Saving in progress for file name - audioData.npy :::
::: Save complete for file name - audioData.npy :::
::: Saving in progress for file name - spectrogramData.npy :::
::: Save complete for file name - spectrogramData.npy :::


TypeError: Missing required positional argument